In [29]:
from collections import defaultdict

from scipy.io import wavfile
from textgrid import TextGrid as TG
from textgrid import Interval
from tqdm import tqdm

import os

In [30]:
root_folder = '../../allwavs/allvowl'

all_files = os.listdir(root_folder)
all_lower_files = [file.lower() for file in all_files]
corpus = defaultdict(list)
final_corpus = {}

variant = True

for file in all_files:
    file_l = file.lower().replace('_1', '').replace('cut', '').replace('-corrected', '')
    if file_l.endswith('wav'):
        corpus[file_l[:-4]].append(file)
    if file_l.endswith('textgrid'):
        corpus[file_l[:-9]].append(file)

for k, v in corpus.items():
    if not all(not x.lower().endswith('textgrid') for x in v) or all(not x.lower().endswith('wav') for x in v):
        wavs = [file for file in v if file.lower().endswith('wav')]
        if len(wavs) > 1:
            wavs = [x for x in wavs if 'cut' in x]
        wav = wavs[0]
        textgrids = [file for file in v if file.lower().endswith('textgrid')]
        if len(textgrids) > 1:
            if variant:
                textgrids = [x for x in textgrids if '_1' in x]
            else:
                textgrids = [x for x in textgrids if 'mono.' in x]
            if k == 'om1_mono':
                textgrids = ['om1_mono.TextGrid']
        textgrid = textgrids[0]
        final_corpus[k] = [wav, textgrid]
    else:
        print(k, v)

dj2_mono ['dj2cut_mono.wav']
kd_mono ['kdcut_mono.wav']
ken1_mono ['ken1cut_mono.wav']
laure_mono ['laurecut_mono.wav']
lp1_mono ['lp1cut_mono.wav']
lp2_mono ['lp2cut_mono.wav']
lucas_mono ['lucas_mono.wav']
mat1_mono ['mat1_mono.wav']
mathieu2_mono ['mathieu2_mono.wav']
melanie1_mono ['melanie1cut_mono.wav']
melanie2_mono ['melanie2cut_mono.wav']
mob_mono ['mob_mono.wav']
or1_mono ['or1_mono.wav']
or2_mono ['or2_mono.wav']
unk1_mono ['unk1cut_mono.wav']
unk2_mono ['unk2cut_mono.wav']
vb1_mono ['vb1_mono.wav']
vibl2_mono ['vibl2cut_mono.wav']
yf1_mono ['yf1cut_mono.wav']
yf2_mono ['yf2cut_mono.wav']
yl_mono ['ylcut_mono.wav']


In [56]:
marks = set()

vowels = {'2', '9', '@', 'E', 'E/', 'O', 'O/', 'U~/', 'a', 'a~', 'e', 'i', 'o', 'o~', 'u', 'y'}

gender_dict = {
    'ac1': 'f',
    'ac2': 'f',
    'agnes1': 'f',
    'agnes2': 'f',
    'al1': 'f',
    'al2': 'f',
    'al': 'f',
    'ang1': 'f',
    'ang2': 'f',
    'be1': 'm',
    'be2': 'm',
    'cg1': 'f',
    'cg2': 'f',
    'df': 'm',
    'em1': 'm',
    'em2': 'm',
    'ev1': 'm',
    'ev2': 'm',
    'hzc1': 'f',
    'hzc2': 'f',
    'ib1': 'f',
    'ib2': 'f',
    'isabel22': 'f',
    'isabel2': 'f',
    'jb1': 'm',
    'jb2': 'm',
    'lb1': 'f',
    'lb2': 'f',
    'om1': 'f',
    'om2': 'f',
    'oriane-re2': 'f',
    'oriane1': 'f',
    'phil1': 'm',
    'phil2': 'm',
    'simon1': 'm',
    'simon2': 'm',
    'vb2': 'f',
    'vc1': 'm',
    'vc2': 'm',
    'vg': 'm',
    'vibl1': 'f',
    'ya1': 'm',
    'ya2': 'm',
}

def trim_wav(in_wav, out_wav, start, end):
    sample_rate, wave_data = wavfile.read(in_wav)
    start_sample = int(start * sample_rate)
    end_sample = int(end * sample_rate)
    wavfile.write(out_wav, sample_rate, wave_data[start_sample:end_sample])

def __hash__(self):
    return hash((self.minTime, self.maxTime, self.mark))

Interval.__hash__ = __hash__

for k, (wav, textgrid) in tqdm(final_corpus.items()):
    try:
        tg = TG.fromFile(root_folder + '/' + textgrid)
    except AttributeError:
        #print(textgrid)
        continue
        
    out_phones = defaultdict(list)
    gender = gender_dict[k.replace('_mono', '')]
    if len(tg) == 2:
        #print(k)
        #print(k.ljust(20), wav.ljust(20), textgrid)
        phontier, wordtier = tg
        for phone in phontier:
            if phone.mark != '#':
                best_overlap_duration = 0
                best_word = None
                for word in wordtier:
                    overlap_duration = max(0, min(phone.maxTime, word.maxTime) - max(phone.minTime, word.minTime))
                    if overlap_duration > best_overlap_duration:
                        best_overlap_duration = overlap_duration
                        best_word = word
                out_phones[best_word].append(phone)
        for word, phones in out_phones.items():
            word2 = word.mark
            if 'One' in word.mark and word.mark.startswith('t'):
                word2 = word2.replace('One', '1')
                phones = phones[:len(prev)]
                for i, phone in enumerate(phones):
                    phone.mark = prev[i].mark
            elif 'Two' in word.mark:
                phones = phones[:2]
                word2 = word2.replace('Two', '0')

            phonemes = '_'.join([phone.mark for phone in phones])
            min_d = phones[0].minTime
            max_d = phones[-1].maxTime

            if phones[1:2]:
                phones = phones[1:2]
            elif phones[0].mark not in vowels:
                continue
            
            phone = phones[0]
            phone.mark = phone.mark.replace('/', '+')
            phonemes = phonemes.replace('/', '+')
            #print(phone.mark, phonemes, word2, wav)
            trim_wav(f'{root_folder}/{wav}', f'{root_folder}/extracted/{phone.mark}__{gender}__{phone.minTime - min_d:.5f}__{phone.maxTime - min_d:.5f}__{phonemes}__{word2}__{wav}', min_d, max_d)
            prev = phones
            for phone in phones:
                phone.mark = phone.mark.replace('+', '/')

  0%|                                                                                                                     | 0/46 [00:00<?, ?it/s]

i p_i pi ac1cut_mono.wav
i l_i lit ac1cut_mono.wav
i m_i mi ac1cut_mono.wav
i s_i si ac1cut_mono.wav
i t_i ti ac1cut_mono.wav
i i ti1 ac1cut_mono.wav
a p_a pas ac1cut_mono.wav
a l_a la ac1cut_mono.wav
a m_a ma ac1cut_mono.wav
a s_a sa ac1cut_mono.wav
a t_a ta ac1cut_mono.wav
a a ta1 ac1cut_mono.wav
u p_u pou ac1cut_mono.wav
u l_u loup ac1cut_mono.wav
u m_u mou ac1cut_mono.wav
u s_u sous ac1cut_mono.wav
u t_u tout ac1cut_mono.wav
u u tout1 ac1cut_mono.wav
e p_e pé ac1cut_mono.wav
E+ l_E+ les ac1cut_mono.wav
E+ m_E+ mes ac1cut_mono.wav
E+ s_E+ ses ac1cut_mono.wav
E+ t_E+ tes ac1cut_mono.wav
E+ E+ tes1 ac1cut_mono.wav
E+ p_E+ paix ac1cut_mono.wav
E l_E lait ac1cut_mono.wav
E m_E mais ac1cut_mono.wav
E s_E sait ac1cut_mono.wav
E+ t_E+ taie ac1cut_mono.wav
E+ E+ taie1 ac1cut_mono.wav
O p_O_R port ac1cut_mono.wav
O l_O_R lors ac1cut_mono.wav
O m_O_R mort ac1cut_mono.wav
O s_O_R sort ac1cut_mono.wav
O t_O_R tort ac1cut_mono.wav
O O tort1 ac1cut_mono.wav
o p_o peau ac1cut_mono.wav
o l_o lot ac

  2%|██▎                                                                                                          | 1/46 [00:02<02:11,  2.91s/it]

a~ t_a~ tant0 ac1cut_mono.wav
a~ t_a~ tant0 ac1cut_mono.wav
i p_i pi ac2cut_mono.wav
i l_i lit ac2cut_mono.wav
i m_i mi ac2cut_mono.wav
i s_i si ac2cut_mono.wav
i t_i ti ac2cut_mono.wav
i i ti1 ac2cut_mono.wav
a p_a pas ac2cut_mono.wav
a l_a la ac2cut_mono.wav
a m_a ma ac2cut_mono.wav
a s_a sa ac2cut_mono.wav
a t_a ta ac2cut_mono.wav
a a ta1 ac2cut_mono.wav
u p_u pou ac2cut_mono.wav
u l_u loup ac2cut_mono.wav
u m_u mou ac2cut_mono.wav
u s_u sous ac2cut_mono.wav
u t_u tout ac2cut_mono.wav
u u tout1 ac2cut_mono.wav
e p_e pé ac2cut_mono.wav
E+ l_E+ les ac2cut_mono.wav
E+ m_E+ mes ac2cut_mono.wav
E+ s_E+ ses ac2cut_mono.wav
E+ t_E+ tes ac2cut_mono.wav
E+ E+ tes1 ac2cut_mono.wav
E+ p_E+ paix ac2cut_mono.wav
E l_E lait ac2cut_mono.wav
E m_E mais ac2cut_mono.wav
E s_E sait ac2cut_mono.wav
E+ t_E+ taie ac2cut_mono.wav
E+ E+ taie1 ac2cut_mono.wav
O p_O_R port ac2cut_mono.wav
O l_O_R lors ac2cut_mono.wav
O m_O_R mort ac2cut_mono.wav
O s_O_R sort ac2cut_mono.wav
O t_O_R tort ac2cut_mono.wav
O O t

  4%|████▋                                                                                                        | 2/46 [00:05<02:09,  2.95s/it]

...

 52%|████████████████████████████████████████████████████████▎                                                   | 24/46 [00:55<00:49,  2.27s/it]

i p_i pi lb2cut_mono.wav
i l_i lit lb2cut_mono.wav
i m_i mi lb2cut_mono.wav
i s_i si lb2cut_mono.wav
i t_i ti lb2cut_mono.wav
i i ti1 lb2cut_mono.wav
a p_a pas lb2cut_mono.wav
a l_a la lb2cut_mono.wav
a m_a ma lb2cut_mono.wav
a s_a sa lb2cut_mono.wav
a t_a ta lb2cut_mono.wav
a a ta1 lb2cut_mono.wav
u p_u pou lb2cut_mono.wav
u l_u loup lb2cut_mono.wav
u m_u mou lb2cut_mono.wav
u s_u sous lb2cut_mono.wav
u t_u tout lb2cut_mono.wav
u u tout1 lb2cut_mono.wav
e p_e pé lb2cut_mono.wav
E+ l_E+ les lb2cut_mono.wav
E+ m_E+ mes lb2cut_mono.wav
E+ s_E+ ses lb2cut_mono.wav
E+ t_E+ tes lb2cut_mono.wav
E+ E+ tes1 lb2cut_mono.wav
E+ p_E+ paix lb2cut_mono.wav
E l_E lait lb2cut_mono.wav
E m_E mais lb2cut_mono.wav
E s_E sait lb2cut_mono.wav
E+ t_E+ taie lb2cut_mono.wav
E+ E+ taie1 lb2cut_mono.wav
O p_O_R port lb2cut_mono.wav
O l_O_R lors lb2cut_mono.wav
O m_O_R mort lb2cut_mono.wav
O s_O_R sort lb2cut_mono.wav
O t_O_R tort lb2cut_mono.wav
O O tort1 lb2cut_mono.wav
o p_o peau lb2cut_mono.wav
o l_o lot lb

 67%|████████████████████████████████████████████████████████████████████████▊                                   | 31/46 [00:58<00:14,  1.05it/s]

a~ m_a~ ment0 lb2cut_mono.wav
a~ s_a~ sans0 lb2cut_mono.wav
a~ t_a~ tant0 lb2cut_mono.wav
a~ t_a~ tant0 lb2cut_mono.wav


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46/46 [00:58<00:00,  1.27s/it]
